# HMM Profile Construction for Kunitz Domain Proteins
**AUTHOR:** Alessia Corica\
**PROJECT:** LAB1 Kunitz project\
**GOAL:** Extract, cluster, and model Kunitz domain sequences from UniProt and PDB to build a profile HMM.

## Step 1: Prepare UniProt and PDB data

- Extract human and non-human Kunitz sequences from `kunitz_sequences.fasta`
- Convert custom PDB CSV into FASTA format to use `cd-hit` in the next step


In [ ]:
# Extract human sequences (Homo sapiens)
awk '/^>/ {f=($0 ~ /OS=Homo sapiens/)} f' kunitz_sequences.fasta > human_kunitz_sequences.fasta

# Extract non-human sequences
grep -v "Homo sapiens" kunitz_sequences.fasta > nothuman_kunitz_sequences.fasta

# Convert PDB CSV to FASTA (PF00014 only)
cat rcsb_pdb_custom_report_20250505025420.csv | tr -d '"' | awk -F ',' '{if (length($2)>0) {name=$2}; print name ,$3,$4,$5}' | grep PF00014 | awk '{print ">"$1"_"$3; print $2}' > pdb_kunitz_customreported.fasta


## Step 2: Cluster sequences and remove redundancy

- Cluster sequences at 90% identity using `cd-hit`
- Filter out sequences not suitable for modeling
- Select one representative sequence per cluster to obtain a non-redundant dataset using `.clstr`


In [ ]:
# Cluster sequences at 90% identity
cd-hit -i pdb_kunitz_customreported.fasta -o pdb_kunitz_customreported_nr.fasta -c 0.9

# Remove outliers (2ODY_E in this case)
awk '/^>2ODY_E/ {getline; next} {print}' pdb_kunitz_customreported_nr.fasta > pdb_kunitz_customreported_filtered.fasta

# Repeat the previous step also for the corresponding .clstr file
awk 'BEGIN{skip=0} /^>Cluster 0$/ {skip=1; next} /^>Cluster/ {skip=0} !skip' pdb_kunitz_customreported_nr.fasta.clstr > pdb_kunitz_customreported_filtered.clstr

# Convert .clstr to readable text file
clstr2txt.pl pdb_kunitz_customreported_filtered.clstr > pdb_kunitz.clusters.txt

# Get representative IDs
awk '$5 == 1 {print $1}' pdb_kunitz.clusters.txt > pdb_kunitz_rp.ids

# Retrieve corresponding sequences
for i in $(cat pdb_kunitz_rp.ids); do \
  grep -A 1 "^>$i" pdb_kunitz_customreported.fasta | head -n 2 >> pdb_kunitz_rp.fasta; \
done


## Step 3: Build the profile HMM

- Format FASTA for HMMER: remove header description and convert sequences to uppercase
- Build the HMM profile using `hmmbuild`


In [ ]:
# Format FASTA for HMMER
awk '{ if (substr($1,1,1)==">") { print "\n" toupper($1) } else { printf "%s", toupper($1) }}' pdb_kunitz_rp.fasta > pdb_kunitz_rp_formatted.ali

# Build HMM profile
hmmbuild structural_model.hmm pdb_kunitz_rp_formatted.ali

## Step 4: Create BLAST database and filter redundancy

- Create a BLAST protein database from `kunitz_sequences.fasta`
- Run BLAST (`blastp`) using the representative sequences as queries
- Detect and remove UniProt sequences highly similar to the training set (≥95% identity, ≥50% coverage)



In [ ]:
# Create BLAST database
makeblastdb -in kunitz_sequences.fasta -dbtype prot -out kunitz_sequences.fasta

# Run BLAST alignment
blastp -query pdb_kunitz_rp.fasta -db kunitz_sequences.fasta -out pdb_kunitz_nr_23.blast -outfmt 7

# Extract matching UniProt IDs with identity ≥95% and coverage ≥50%
grep -v "^#" pdb_kunitz_nr_23.blast | awk '{if ($3>=95 && $4>=50) print $2}' | sort -u | cut -d "|" -f 2 > to_remove.ids

# Get all original UniProt IDs from FASTA
grep ">" kunitz_sequences.fasta | cut -d "|" -f 2 > all_kunitz.id

# Remove redundant IDs
comm -23 <(sort all_kunitz.id) <(sort to_remove.ids) > to_keep.ids


## Step 5: Extract final non-redundant positive sequences

- Use the list of non-redundant UniProt IDs (`to_keep.ids`) to extract the corresponding sequences from `kunitz_sequences.fasta`.
- This produces `ok_kunitz.fasta`, the final FASTA file containing only the filtered Kunitz domain proteins.




In [ ]:
# Extract non-redundant Kunitz sequences
python3 get_seq.py to_keep.ids kunitz_sequences.fasta ok_kunitz.fasta

## Step 6: Create the negative set

- Extract all UniProt IDs from `uniprot_sprot.fasta`
- Remove IDs known to contain a Kunitz domain (stored in `all_kunitz.id`)
- Save the remaining IDs to `sp_negs.ids` and extract the sequences using `get_seq.py`

In [ ]:
# Extract all UniProt IDs from SwissProt FASTA
grep ">" uniprot_sprot.fasta | cut -d"|" -f2 > sp.id

# Remove IDs with Kunitz domain (PF00014)
comm -23 <(sort sp.id) <(sort all_kunitz.id) > sp_negs.ids

# Extract final negative sequences
python3 get_seq.py sp_negs.ids uniprot_sprot.fasta sp_negs.fasta


## Step 7: Create train/test splits

- Split positive and negative IDs into train/test halves for evaluation using `head`/`tail`
- Generate corresponding FASTA files using `get_seq.py`

In [ ]:
# Shuffle IDs randomly
sort -R to_keep.ids > random_ok_kunitz.ids
sort -R sp_negs.ids > random_sp_negs.ids

# Positive set (total: 366 IDs)
head -n 183 random_ok_kunitz.ids > pos_1.ids
tail -n 183 random_ok_kunitz.ids > pos_2.ids

# Negative set (total: 572834 IDs)
head -n 286417 random_sp_negs.ids > neg_1.ids
tail -n 286417 random_sp_negs.ids > neg_2.ids

# Extract sequences
python3 get_seq.py pos_1.ids uniprot_sprot.fasta pos_1.fasta
python3 get_seq.py pos_2.ids uniprot_sprot.fasta pos_2.fasta
python3 get_seq.py neg_1.ids uniprot_sprot.fasta neg_1.fasta
python3 get_seq.py neg_2.ids uniprot_sprot.fasta neg_2.fasta

## Step 8: Run `hmmsearch` and normalize

- Run `hmmsearch` on each test set
- Repeat with `-Z 1000` to normalize E-values across datasets

In [ ]:
# Run hmmsearch on positive and negative sets
hmmsearch --max --tblout pos_1.out structural_model.hmm pos_1.fasta
hmmsearch --max --tblout pos_2.out structural_model.hmm pos_2.fasta
hmmsearch --max --tblout neg_1.out structural_model.hmm neg_1.fasta
hmmsearch --max --tblout neg_2.out structural_model.hmm neg_2.fasta

# Repeat hmmsearch with -Z 1000 to normalize E-values across datasets
hmmsearch -Z 1000 --max --tblout pos_1.out structural_model.hmm pos_1.fasta
hmmsearch -Z 1000 --max --tblout pos_2.out structural_model.hmm pos_2.fasta
hmmsearch -Z 1000 --max --tblout neg_1.out structural_model.hmm neg_1.fasta
hmmsearch -Z 1000 --max --tblout neg_2.out structural_model.hmm neg_2.fasta


## Step 9: Parse `hmmsearch` outputs and build `.class` files

- Build `.class` files with ID, label, score, and E-value
- Add sequences not matched by `hmmsearch`

In [ ]:
# Extract matched hits from hmmsearch outputs and format as .class
grep -v "^#" pos_1.out | awk '{split($1,a,"|"); print a[2]"\t1\t"$5"\t"$8}' > pos_1.class
grep -v "^#" pos_2.out | awk '{split($1,a,"|"); print a[2]"\t1\t"$5"\t"$8}' > pos_2.class

grep -v "^#" neg_1.out | awk '{split($1,a,"|"); print a[2]"\t0\t"$5"\t"$8}' > neg_1.class
grep -v "^#" neg_2.out | awk '{split($1,a,"|"); print a[2]"\t0\t"$5"\t"$8}' > neg_2.class

# Add missing negatives
comm -23 <(sort neg_1.ids) <(cut -f1 neg_1.class | sort) | awk '{print $1"\t0\t10.0\t10.0"}' >> neg_1.class
comm -23 <(sort neg_2.ids) <(cut -f1 neg_2.class | sort) | awk '{print $1"\t0\t10.0\t10.0"}' >> neg_2.class

# Combine positive and negative entries into final test sets
cat pos_1.class neg_1.class > set_1.class
cat pos_2.class neg_2.class > set_2.class

## Step 10: Evaluate model performance

- Evaluate the model using the `performance.py` script using a classification threshold of `1e-5`
- Output metrics include:
  - Accuracy
  - Sensitivity
  - Specificity


In [ ]:
# Run performance script on both sets
python3 performance.py set_1.class 1e-5
python3 performance.py set_2.class 1e-5

## Step 12: Threshold exploration

- Evaluate model performance on thresholds 1e-1 to 1e-10
- Extract false negatives and false positives for each set

In [ ]:
# Evaluate performance across thresholds for each set
for i in $(seq 1 10); do
    python3 performance.py set_1.class 1e-$i
done > performance_set1.txt

for i in $(seq 10 -1 1); do
    python3 performance.py set_2.class 1e-$i
done > performance_set2.txt

# Extract false negatives (e-value > 1e-5)
awk '$2 == 1 && $3 > 1e-5' pos_1.class | sort -grk 3 > fn_pos1.txt
awk '$2 == 1 && $3 > 1e-5' pos_2.class | sort -grk 3 > fn_pos2.txt

# Extract false positives (e-value < 1e-5)
awk '$2 == 0 && $3 < 1e-5' neg_1.class | sort -grk 3 > fp_neg1.txt
awk '$2 == 0 && $3 < 1e-5' neg_2.class | sort -grk 3 > fp_neg2.txt